SDN OFAC csv file

In [2]:
import pandas as pd
import json
import re

file_path = '../original/sdn.csv'
df1 = pd.read_csv(file_path)
print(df1.columns)
df1.head()

Index(['ent_num', 'SDN_name', 'SDN_type', 'Program', 'Title', 'Call_Sign',
       'Vess_type', 'Tonnage', 'GRT', 'Vess_flag', 'Vess_owner', 'Remarks'],
      dtype='object')


,ent_num,SDN_name,SDN_type,Program,Title,Call_Sign,Vess_type,Tonnage,GRT,Vess_flag,Vess_owner,Remarks
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-


In [3]:
# COUNTRY

# read add.csv which is the helper file to add the country column to sdn.csv
add_file_path = '../original/add.csv'
df2 = pd.read_csv(add_file_path)
df2 = df2[['ent_num', 'country']]
if 'country' in df1.columns:
    df1.drop(columns=['country'], inplace=True)

# add a new column country in sdn.csv and add the values from add.csv when ent_num matches then concatanate the values
merged_df = pd.merge(df1, df2[['ent_num', 'country']], on='ent_num', how='left')
grouped_df = merged_df.groupby('ent_num')['country'].apply(lambda x: ';'.join(x.dropna())).reset_index()
result_df = pd.merge(df1, grouped_df, on='ent_num', how='left')
result_df.columns


Index(['ent_num', 'SDN_name', 'SDN_type', 'Program', 'Title', 'Call_Sign',
       'Vess_type', 'Tonnage', 'GRT', 'Vess_flag', 'Vess_owner', 'Remarks',
       'country'],
      dtype='object')

In [4]:
col = ['Program', 'Title', 'Call_Sign',
       'Vess_type', 'Tonnage', 'GRT', 'Vess_flag', 'Vess_owner']
columns_to_drop = [col_name for col_name in col if col_name in result_df.columns]
result_df.drop(columns=columns_to_drop, inplace=True)
result_df.columns

Index(['ent_num', 'SDN_name', 'SDN_type', 'Remarks', 'country'], dtype='object')

In [5]:
print(result_df['SDN_type'].value_counts())
result_df = result_df[result_df['SDN_type'] == 'individual']
print("Cleaned",result_df['SDN_type'].value_counts())
result_df.drop(columns="SDN_type", inplace=True)

-0-           7252
individual    6915
vessel         861
aircraft       374
Name: SDN_type, dtype: int64
Cleaned individual    6915
Name: SDN_type, dtype: int64


Individual have proper names and some vessels are right name, but most of them are not. Aircrafts seem to be aircraft manufacturers and have codes in names, hence not of any use. -0- seems to be names of company or groups, hence disregarded.

In [6]:
len(result_df) #total individuals

6915

In [7]:
# DOB

result_df['Remarks'] = result_df['Remarks'].str.lower()                         
# for format dd mmm yyyy
result_df['DOB'] = result_df['Remarks'].str.extract(r'(\d{2} \w{3} \d{4})')
result_df['day'] = result_df['DOB'].str.extract(r'(\d{2})')
result_df['month'] = result_df['DOB'].str.extract(r'(\w{3})')
result_df['year'] = result_df['DOB'].str.extract(r'(\d{4})')
# for yyyy only format
result_df['year'] = result_df['Remarks'].str.extract(r'(\d{4})')
result_df.head()

print(result_df['year'].count()) # total individuals with at least year in dob
print(result_df['DOB'].count()) # total individuals with whole dob


6855
5924


In [8]:
# GENDER
def extract_gender(text):
    pattern = r'gender (male|female)'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
# Apply the function to extract the gender
result_df["Gender"] = result_df['Remarks'].apply(extract_gender)
result_df.head()
result_df["Gender"].count() 
# result_df[result_df['ent_num'] == "12610"]

4325

In [9]:
result_df[result_df['ent_num'] == "12599"]

,ent_num,SDN_name,Remarks,country,DOB,day,month,year,Gender
2096,12599,"PATEK, Umar","dob 20 jul 1966; pob central java, indonesia; ...",-0-,20 jul 1966,20,jul,1966,None


In [10]:
def extract_nationality(remark):
    pattern = r'nationality ([A-Za-z]+);'
    match = re.search(pattern, remark)
    if match:
        return match.group(1)
    else:
        return None

def extract_citizen(remark):
    pattern = r'citizen ([A-Za-z]+);'
    match = re.search(pattern, remark)
    if match:
        return match.group(1)
    else:
        return None


# Apply the extract_nationality function to the 'remarks' column
result_df['Citizen'] = result_df['Remarks'].apply(extract_citizen)
result_df['Nationality'] = result_df['Remarks'].apply(extract_nationality)
result_df['Nationality'].count()

filtered_df = result_df.dropna(subset=['Citizen', 'Nationality'])
diff_values_df = filtered_df[filtered_df['Citizen'] != filtered_df['Nationality']]
count_diff_values = diff_values_df.shape[0]
print(count_diff_values) # 20 instances where in remark both citizen <country1> and nationality <country2> are mentioned, hence seperated


20


In [11]:
pattern = r'passport ([^\(]+) \(([^)]+)\)'

def extract_passport_info(remark):
    match = re.search(pattern, remark)
    if match:
        return match.group(1), match.group(2)
    else:
        return None, None


result_df[['Pass_No', 'Pass_Country']] = result_df['Remarks'].apply(lambda x: pd.Series(extract_passport_info(x)))
result_df['Pass_No'].count() # total individuals with passport number

1549

In [12]:
pattern = r'eth (0x[0-9a-fA-F]+);'

def extract_eth_address(remark):
    match = re.search(pattern, remark)
    if match:
        return match.group(1)
    else:
        return None

result_df['Eth_address'] = result_df['Remarks'].apply(extract_eth_address)

# Filter rows where 'eth_address' is not null
filtered_df = result_df.dropna(subset=['Eth_address'])
filtered_df['Eth_address'].count() # total individuals with eth address

11

In [13]:
result_df.columns

Index(['ent_num', 'SDN_name', 'Remarks', 'country', 'DOB', 'day', 'month',
       'year', 'Gender', 'Citizen', 'Nationality', 'Pass_No', 'Pass_Country',
       'Eth_address'],
      dtype='object')

In [14]:
# the sdn_name column has in the format "LAST NAME, first name", so split it up into 2 columns if you find the comma in the string
result_df[['Last_Name', 'First_Name']] = result_df['SDN_name'].str.split(', ', expand=True, n=1)
result_df['Last_Name'] = result_df['Last_Name'].str.upper()
result_df['First_Name'] = result_df['First_Name'].str.upper()

result_df.head()

,ent_num,SDN_name,Remarks,country,DOB,day,month,year,Gender,Citizen,Nationality,Pass_No,Pass_Country,Eth_address,Last_Name,First_Name
53,2674,"ABBAS, Abu",dob 10 dec 1948; director of palestine liberat...,-0-,10 dec 1948,10,dec,1948,None,None,None,None,None,None,ABBAS,ABU
54,2675,"AL RAHMAN, Shaykh Umar Abd",dob 03 may 1938; pob egypt; chief ideological ...,-0-,03 may 1938,03,may,1938,None,None,None,None,None,None,AL RAHMAN,SHAYKH UMAR ABD
55,2676,"AL ZAWAHIRI, Dr. Ayman","dob 19 jun 1951; pob giza, egypt; passport 108...",-0-,19 jun 1951,19,jun,1951,None,None,None,1084010,egypt,None,AL ZAWAHIRI,DR. AYMAN
56,2677,"AL-ZOMOR, Abboud Abdul Latif Hassan","dob 19 apr 1947; pob nahia, giza, egypt; natio...",Egypt,19 apr 1947,19,apr,1947,male,None,egypt,None,None,None,AL-ZOMOR,ABBOUD ABDUL LATIF HASSAN
57,2678,"AWDA, Abd Al Aziz",dob 1946; chief ideological figure of palestin...,-0-,NaN,NaN,NaN,1946,None,None,None,None,None,None,AWDA,ABD AL AZIZ


In [15]:
result_df.to_csv('cleaned_sdn.csv', index=False)
result_df = result_df.where(pd.notnull(result_df), None)

filtered_df = result_df.dropna(subset=['Pass_No', 'Pass_Country'], how='all')
passport_df = filtered_df[['Pass_No', 'Pass_Country']]
filtered_df = result_df.dropna(subset=['Eth_address'], how='all')
eth_df = filtered_df[['Eth_address']]
name_dob = result_df[['First_Name', 'Last_Name', 'day', 'month', 'year']]

passport_list = passport_df.to_dict(orient='records')
with open('passports.json', 'w') as json_file:
    json.dump(passport_list, json_file, indent=4)

etherum_list = eth_df.to_dict(orient='records')
with open('etherum_add.json','w') as json_file:
    json.dump(etherum_list, json_file, indent=4)

name_list = name_dob.to_dict(orient='records')
with open('names.json', 'w') as json_file:
    json.dump(name_list, json_file, indent=4)